In [12]:
from configparser import ConfigParser

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

In [53]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spptify:artist:2WX2uTcsvV5OnS0inACecP'
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

results = sp.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

album_ids = []
for album in albums:
    album_ids.append(album['id'])

track_ids = []
for album in album_ids:

    results = sp.album_tracks(album)
    tracks = results['items']

    
    for track in tracks:
        track_ids.append(track['id'])

track_ids = track_ids[0:10]
track_ids


['2mpezzl8FlxttNlaUSDzuV',
 '6VLEEyg7auxTMixCT8U3Gk',
 '0HtdR1K0cptJbOF3EKlAbT',
 '3QorTmBB8lYdSO9bGglOrb',
 '53vcgIyHUsxpeZjcEymEmh',
 '0TVYFZCASBjQrrPWBdyWHY',
 '3L5Vn1Mxq8x0EESC2BleaC',
 '5HsW6nVD3RJ4K7su9HyBGW',
 '5x8apEOj9Q0oxCmjIPK6qt',
 '31iO9bmNknTGoYvXkHjxPe']

In [85]:
import pandas as pd

features = []
for track in track_ids:
    results = sp.audio_features(track)
    features.append(results[0])

# print(features)

data = pd.DataFrame.from_records(features)
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.364,0.0454,5,-26.645,1,0.0437,0.958,0.985000,0.1220,0.0528,111.013,audio_features,2mpezzl8FlxttNlaUSDzuV,spotify:track:2mpezzl8FlxttNlaUSDzuV,https://api.spotify.com/v1/tracks/2mpezzl8Flxt...,https://api.spotify.com/v1/audio-analysis/2mpe...,49507,5
1,0.349,0.4720,5,-12.792,1,0.0485,0.694,0.005680,0.1550,0.4610,88.167,audio_features,6VLEEyg7auxTMixCT8U3Gk,spotify:track:6VLEEyg7auxTMixCT8U3Gk,https://api.spotify.com/v1/tracks/6VLEEyg7auxT...,https://api.spotify.com/v1/audio-analysis/6VLE...,266480,4
2,0.595,0.5600,1,-7.988,0,0.0327,0.536,0.000000,0.1560,0.4830,131.833,audio_features,0HtdR1K0cptJbOF3EKlAbT,spotify:track:0HtdR1K0cptJbOF3EKlAbT,https://api.spotify.com/v1/tracks/0HtdR1K0cptJ...,https://api.spotify.com/v1/audio-analysis/0Htd...,202400,4
3,0.577,0.4150,9,-10.550,1,0.0514,0.825,0.000002,0.0887,0.4630,91.949,audio_features,3QorTmBB8lYdSO9bGglOrb,spotify:track:3QorTmBB8lYdSO9bGglOrb,https://api.spotify.com/v1/tracks/3QorTmBB8lYd...,https://api.spotify.com/v1/audio-analysis/3Qor...,206840,4
4,0.353,0.3860,0,-10.927,1,0.0450,0.661,0.000231,0.0806,0.2690,147.808,audio_features,53vcgIyHUsxpeZjcEymEmh,spotify:track:53vcgIyHUsxpeZjcEymEmh,https://api.spotify.com/v1/tracks/53vcgIyHUsxp...,https://api.spotify.com/v1/audio-analysis/53vc...,234240,4
5,0.492,0.2140,1,-13.206,1,0.0534,0.960,0.000072,0.1070,0.2800,93.722,audio_features,0TVYFZCASBjQrrPWBdyWHY,spotify:track:0TVYFZCASBjQrrPWBdyWHY,https://api.spotify.com/v1/tracks/0TVYFZCASBjQ...,https://api.spotify.com/v1/audio-analysis/0TVY...,297520,5
6,0.493,0.4510,11,-10.050,0,0.0315,0.462,0.000171,0.0937,0.3530,87.156,audio_features,3L5Vn1Mxq8x0EESC2BleaC,spotify:track:3L5Vn1Mxq8x0EESC2BleaC,https://api.spotify.com/v1/tracks/3L5Vn1Mxq8x0...,https://api.spotify.com/v1/audio-analysis/3L5V...,225733,4
7,0.537,0.3470,8,-9.732,1,0.0303,0.523,0.000000,0.1120,0.2990,76.999,audio_features,5HsW6nVD3RJ4K7su9HyBGW,spotify:track:5HsW6nVD3RJ4K7su9HyBGW,https://api.spotify.com/v1/tracks/5HsW6nVD3RJ4...,https://api.spotify.com/v1/audio-analysis/5HsW...,248133,4
8,0.387,0.4320,2,-8.870,0,0.0370,0.577,0.000037,0.0802,0.0925,102.771,audio_features,5x8apEOj9Q0oxCmjIPK6qt,spotify:track:5x8apEOj9Q0oxCmjIPK6qt,https://api.spotify.com/v1/tracks/5x8apEOj9Q0o...,https://api.spotify.com/v1/audio-analysis/5x8a...,232080,3
9,0.611,0.4120,2,-10.395,1,0.0526,0.739,0.000159,0.1200,0.4490,94.924,audio_features,31iO9bmNknTGoYvXkHjxPe,spotify:track:31iO9bmNknTGoYvXkHjxPe,https://api.spotify.com/v1/tracks/31iO9bmNknTG...,https://api.spotify.com/v1/audio-analysis/31iO...,213187,4
